In [14]:
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score

In [15]:
RANDOM_SEED = 13

In [16]:
df = pd.read_csv("exemplos.csv", sep=",", index_col=0)
df.head()

,Intenção,Sentença
0,Consultar saldo da conta,Qual o meu saldo
1,Interagir com a luz ou o ar-condicionado,Ligue a luz
2,Consultar saldo da conta,Me diga meu saldo
3,Consultar saldo da conta,Qual o meu saldo?
4,Interagir com a luz ou o ar-condicionado,está muito claro


## Limpando o dataset

In [18]:
df.Sentença = df["Sentença"].str.lower()

## Adequação dos dados para o MultinomialNB

In [19]:
# Adequa os dados para entrada no modelo MultinomialNB e já separa em X e y.
v = CountVectorizer()
def adaptToModel(df, objective=None, vectorizer=v):
    txts = df["Sentença"].tolist()
    if objective == "train":
        counts = vectorizer.fit_transform(txts)
        return counts, df["Intenção"]
    elif objective == "test":
        counts = vectorizer.transform(txts)
        return counts, df["Intenção"]
    else:
        raise ValueError("Defina o objetivo ('train' ou 'test')")

## Separando o dataset em treinamento e teste

In [20]:
# Divide em treinamento e teste, de modo estratificado, o conjunto de dados.
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=RANDOM_SEED)
for train_index, test_index in split.split(df, df["Intenção"]):
    strat_train_set = df.loc[train_index]
    strat_test_set = df.loc[test_index]

## Treinando o modelo

In [21]:
model = MultinomialNB()
train_data = adaptToModel(strat_train_set, objective="train")
model.fit(train_data[0], train_data[1])


MultinomialNB()

## Testando o modelo

In [22]:
test_data = adaptToModel(strat_test_set, objective="test")

In [23]:
pred = model.predict(test_data[0])

In [24]:
output = pd.DataFrame(columns=["Real", "Predicted"])
output["Real"] = test_data[1]
output["Predicted"] = pred
output.head()

,Real,Predicted
58,Obter informações relativas ao clima,Obter informações relativas ao clima
112,Obter informações relativas ao clima,Obter informações relativas ao clima
31,Consultar saldo da conta,Consultar saldo da conta
79,Interagir com a luz ou o ar-condicionado,Interagir com a luz ou o ar-condicionado
118,Obter informações relativas ao clima,Obter informações relativas ao clima


In [26]:
score = accuracy_score(output["Real"], output["Predicted"])
print(f"Acurácia: {score * 100}%")

Acurácia: 92.0%
